In [1]:
import matplotlib as mlt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from random import choice


### Итерационный метод Гаусса-Зейделя


In [430]:
def seidel(A, b, eps):
    L_inv = np.linalg.inv(np.tril(A))
    U = np.triu(A, 1)
    x = np.zeros(b.shape[0])

    while np.linalg.norm(A @ x - b) > eps:
        x = L_inv @ (b - U @ x)
    return x


### Генератор матриц с диагональным преобладанием


In [3]:
def gen_matrix(n, k):
    a = np.zeros((n, n))
    pos_ij = [0, -1, -2, -3, -4]
    for i in range(n):
        for j in range(n):
            if i == j: continue
            a[i, j] = choice(pos_ij)
        a[i, i] = -np.sum(a[i]) if i > 0 else -np.sum(a[i]) + 10**(-k)
    return a


### Генератор матриц Гильберта


In [4]:
def gen_hilbert(n):
    a = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            a[i, j] = 1 / (i + j + 1)
    return a


### Тест


In [424]:
A = np.array(np.mat('16 3; 7 -11'))
b = np.array([11, 13]).T

seidel(A, b, 0.000001)

2.2329545454545467
0.2664320764462822
0.03179019093961344
0.00379314778256834
0.0004525914967850042
5.400239450281674e-05
6.443467524874791e-06
7.68822829400051e-07


array([ 0.81218278, -0.66497459])

In [434]:
size = 300
A_k = gen_matrix(size, 0.01)
F_k = A_k @ np.arange(1, A_k.shape[0] + 1).T

In [435]:
try:
    print(seidel(A_k, F_k, 0.001))
except np.linalg.LinAlgError:
    print('Singular!')

[  0.98494985   1.98492602   2.9849259    3.98492602   4.98492602
   5.98492606   6.98492602   7.9849259    8.98492597   9.98492607
  10.9849259   11.98492606  12.98492606  13.98492607  14.98492602
  15.98492594  16.98492598  17.98492607  18.98492599  19.98492604
  20.98492595  21.98492606  22.98492599  23.98492603  24.98492598
  25.98492595  26.98492591  27.98492603  28.984926    29.984926
  30.984926    31.98492603  32.98492595  33.98492599  34.98492608
  35.984926    36.98492592  37.98492608  38.98492609  39.98492601
  40.98492596  41.98492603  42.98492608  43.98492596  44.98492592
  45.98492606  46.98492608  47.984926    48.98492592  49.98492597
  50.98492592  51.98492592  52.98492605  53.984926    54.98492604
  55.984926    56.98492593  57.98492605  58.98492608  59.98492598
  60.98492597  61.98492606  62.98492594  63.98492605  64.98492606
  65.98492605  66.98492601  67.98492597  68.98492605  69.98492593
  70.98492602  71.98492606  72.98492602  73.98492602  74.98492598
  75.9849259